# Jugar Galaxian con Agente Dueling DQN

## Librerias

In [1]:
import torch
import numpy as np

from gym_envs import make_galaxian_env
from models import DuelingDQN
from policies import DQNPolicy
from recording import record_episode

from pathlib import Path
from IPython.display import Video, display

### Configurar Torch con CUDA

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU


## Carga de Entorno Base

In [3]:
# Crear un entorno con los mismos wrappers que en entrenamiento
#   para conocer las dimensiones de la observación
tmp_env = make_galaxian_env(
    render_mode=None,
    deepmind_wrappers=True,  # AtariPreprocessing + FrameStackObservation
    frame_stack=4,
)

obs, _ = tmp_env.reset()

# Convertir la observación a un array de NumPy
arr = np.array(obs, copy=False)

# Asegurarse de que la forma sea (C, H, W)
if arr.ndim == 3:
    # Casos típicos:
    # - (H, W, C)
    # - (C, H, W)
    if arr.shape[0] == 84 and arr.shape[1] == 84:
        # (H, W, C) -> (C, H, W)
        arr = np.transpose(arr, (2, 0, 1))
elif arr.ndim == 2:
    # 1 frame (H, W) -> (1, H, W)
    arr = arr[None, ...]
else:
    raise ValueError(f"Obs shape inesperado: {arr.shape}")

# Obtener dimensiones
in_channels = arr.shape[0]
num_actions = tmp_env.action_space.n
tmp_env.close()

print(f"in_channels={in_channels}, num_actions={num_actions}")

in_channels=4, num_actions=6


## Reconstruir modelo y cargar pesos

In [4]:
ModelClass = DuelingDQN

model = ModelClass(in_channels=in_channels, num_actions=num_actions).to(device)

# Ruta del checkpoint (ajusta si usas otro archivo)
checkpoint_path = "../models/dueling_dqn_final.pt"

state = torch.load(checkpoint_path, map_location=device)

# Soportar tanto:
# - archivos que guardan solo state_dict
# - archivos tipo checkpoint con clave "model_state_dict"
if isinstance(state, dict) and "model_state_dict" in state:
    model.load_state_dict(state["model_state_dict"])
else:
    model.load_state_dict(state)

model.eval()
print(f"Modelo cargado desde: {checkpoint_path}")

Modelo cargado desde: ../models/dueling_dqn_final.pt


### Crear la policy (epsilon greedy)

In [5]:
policy = DQNPolicy(model=model, device=device, epsilon=0.01)

## Grabar 3 episodios en video

In [6]:
video_paths = []
scores = []

for i in range(3):
    print(f"\n=== Grabando episodio {i+1} ===")
    path = record_episode(
        policy=policy,
        deepmind_wrappers=True,
    )
    video_paths.append(Path(path))

    # El nombre del archivo es: email_timestamp_score.mp4
    stem = video_paths[-1].stem           # sin extensión
    parts = stem.split("_")
    score_str = parts[-1]                 # último elemento = score
    try:
        score = int(score_str)
    except ValueError:
        score = None
        print(f"  [WARN] No se pudo parsear score desde '{stem}'")
    scores.append(score)

print("\nScores de los 3 episodios:")
for i, (p, s) in enumerate(zip(video_paths, scores), start=1):
    print(f"  Episodio {i}: score = {s}, video = {p}")


=== Grabando episodio 1 ===
[INFO] Iniciando grabación de episodio...
[INFO] Creando entorno ALE/Galaxian-v5...
[INFO] Reiniciando entorno Galaxian...
[INFO] Iniciando episodio...
  - Progreso: 500 pasos, recompensa parcial = 1010
  - Progreso: 1000 pasos, recompensa parcial = 1480
  - Progreso: 1500 pasos, recompensa parcial = 2120
[INFO] Episodio finalizado. Pasos: 1720, Recompensa total: 2580
[INFO] Guardando video (1721 frames, escala x1.0)...
[INFO] Video guardado en: videos\lei21752_202511201642_2580.mp4

=== Grabando episodio 2 ===
[INFO] Iniciando grabación de episodio...
[INFO] Creando entorno ALE/Galaxian-v5...
[INFO] Reiniciando entorno Galaxian...
[INFO] Iniciando episodio...
  - Progreso: 500 pasos, recompensa parcial = 750
[INFO] Episodio finalizado. Pasos: 606, Recompensa total: 810
[INFO] Guardando video (607 frames, escala x1.0)...
[INFO] Video guardado en: videos\lei21752_202511201642_810.mp4

=== Grabando episodio 3 ===
[INFO] Iniciando grabación de episodio...
[INF

### Mostrar el video del mejor de los 3 episodios

In [7]:
# Filtrar episodios que sí tienen score parseado
valid_indices = [i for i, s in enumerate(scores) if s is not None]

if not valid_indices:
    print("No se pudieron obtener scores válidos de los nombres de archivo.")
else:
    # Índice del mejor episodio
    best_idx = max(valid_indices, key=lambda i: scores[i])
    best_score = scores[best_idx]
    best_path = video_paths[best_idx]

    print(f"Mejor episodio: #{best_idx+1} con score = {best_score}")
    print(f"Video: {best_path}")

    # Mostrar el video embebido en el notebook
    display(Video(filename=str(best_path), embed=True))

Mejor episodio: #1 con score = 2580
Video: videos\lei21752_202511201642_2580.mp4
